In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from scipy.stats import skew

pd.options.display.max_columns = 500

Снова потренируемся в предсказании цен на недвижимость из [очередного датасета с каггла](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/)! В качестве основной метрики для валидации моделей будем использовать, как и ранее, `MSLE`.

P.S. в данной домашней работе при построении любых моделей, использующих недетерменированные элементы (как бутстрап), в алгоритме указывайте параметр `random_state = 1` для воспроизводимости результатов.

In [ ]:
df = (
    pd.read_csv('data.csv')
    .drop('Id', axis=1)
)

df

In [ ]:
### Разделим выборку на объекты-таргеты

y = df['SalePrice']
X = df.drop(columns=['SalePrice'])

### Логарифмируем таргет для будущей оптимизации
### MSLE через MSE

log_target = np.log1p(y)

In [ ]:
### Это позволяет получить нормальное распределение таргета
### Важно, например, для построения корректной
### С точки зрения статистических свойств
### Линейной модели.
### Хотя здесь мы будем строить ансамбли деревьев,
### И это не особо интересно.

sns.set(rc={'figure.figsize':(15,10)})

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
sns.distplot(y, bins=50)
plt.title('Original Data')
plt.xlabel('Sale Price')

plt.subplot(1,2,2)
sns.distplot(log_target, bins=50)
plt.title('Natural Log of Data')
plt.xlabel('Natural Log of Sale Price')
plt.tight_layout()

### В начале поработаем с пропусками!

Если в какой-либо колонке оказывается достаточно много пропусков, обычно советуют от них избавляться. Мотивировано это тем, что в таких фичах мы можем наблюдать серьезный недостаток информативности, а заполнение пропусков может лишь внести лишнего шума в данные.

Избавьтесь от всех колонок, в которых пропусков оказывается больше 15%. (1б)

In [ ]:
### Можно воспользоваться такой компактной конструкцией

X = X.dropna(axis=1, thresh = 0.85*X.shape[0])

Вещественные колонки заполните медианным значением по фиче, а категориальные - самой популярной по колонке категорией. (2б)

Заметьте, что колонки `MoSold`, `YrSold`, `GarageYrBlt`, `YearBuilt`, `YearRemodAdd` хоть в таблице не являются типами `object`, вряд ли их справедливо использовать как вещественные. Переведите все значения внутри в строки.

In [ ]:
unwanted_num_cols = ['MoSold', 'YrSold', 'GarageYrBlt', 'YearBuilt', 'YearRemodAdd']

### Выделим категориальные фичи

cat_cols = list(X.select_dtypes(include='object').columns)
cat_cols += unwanted_num_cols

X[cat_cols] = df[cat_cols].astype('object')

### Выделим вещественные фичи

num_cols = list(X.select_dtypes(exclude='object').columns)
num_cols = [ele for ele in num_cols if ele not in unwanted_num_cols]

### Заполним пропуски как и хотели!

X[num_cols] = X[num_cols].fillna(X[num_cols].median())
X[cat_cols] = X[cat_cols].fillna(X[cat_cols].mode().iloc[0])

In [ ]:
### Отложенная выборка

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    log_target,
                                                    test_size=0.25,
                                                    random_state=1)

Напишите трансформер, который будет делать следующее:

1. Масштабирование через StandardScaler для вещественных колонок
2. Кодирование через OneHotEncoder для категориальных, содержащих менее, чем 5 уникальных значений
3. Кодирование через TargetEncoder для всех остальных категориальных

Для этого советуем воспользоваться библиотекой `category_encoders` помимо `sklearn`.

А так же классом `ColumnTransformer` из `sklearn.compose`.

P.S. Напомним, что для деревьев процедура StandardScaling не обязательна (решающие деревья нечувствительны к масштабу). Тем не менее, это может сделать обучение модели менее тяжелым (хранить большие числа сложно для задач с большим количеством данных).

In [ ]:
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from sklearn.preprocessing import StandardScaler

cols_for_ohe = [x for x in cat_cols if X[x].nunique() < 5]
cols_for_mte = [x for x in cat_cols if X[x].nunique() >= 5]

cols_for_ohe_idx = [list(X.columns).index(col) for col in cols_for_ohe]
cols_for_mte_idx = [list(X.columns).index(col) for col in cols_for_mte]
numeric_cols_idx = [list(X.columns).index(col) for col in num_cols]

t = [('OneHotEncoder', OneHotEncoder(), cols_for_ohe_idx),
     ('MeanTargetEncoder', TargetEncoder(), cols_for_mte_idx),
     ('StandardScaler', StandardScaler(), numeric_cols_idx)]

col_transform = ColumnTransformer(transformers=t)

Посмотрите, как на наших данных справляется одно Решающее Дерево с дефолтными гиперпараметрами. Добавьте написанный ранее трансформер в модель. (1б)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipe_dt = Pipeline([("column_transformer",
                     col_transform),

                    ("decision_tree",
                     DecisionTreeRegressor())])

pipe_dt.fit(X_train, y_train)

In [ ]:
train_preds = pipe_dt.predict(X_train)
test_preds = pipe_dt.predict(X_test)

train_error = np.mean((train_preds - y_train)**2)
test_error = np.mean((test_preds - y_test)**2)


print(f"Качество на трейне: {train_error}")
print(f"Качество на тесте: {test_error}")

Справляется даже без контроля переобучения!

Посмотрим на перформанс Случайного Леса! Подберите параметры по отложенной выборке по данной сетке `param_grid`. Помните, что подбирать количество деревьев не супер обязательно, достаточно поставить их побольше. Что произошло с качеством модели по сравнению с одиноким деревом? (2б)

In [ ]:
np.random.seed = 1

param_grid = {
    "random_forest__max_depth": [10, 15, 20],
    "random_forest__min_samples_split": [2, 5, 10],
    "random_forest__min_samples_leaf": [1, 3, 5]
}

custom_cv = [(X_train.index.to_list(), X_test.index.to_list())]

pipe_rf = Pipeline([("column_transformer",
                     col_transform),

                    ("random_forest",
                     RandomForestRegressor(random_state=1))])

search = GridSearchCV(pipe_rf,
                      param_grid,
                      cv=custom_cv,
                      scoring='neg_mean_squared_error',
                      verbose=10)

search.fit(X, log_target)

print(f"Best parameter (CV score={search.best_score_:.5f}):")
print(search.best_params_)

Посмотрим на лучшие параметры:

In [ ]:
search.best_params_

In [ ]:
pipe_rf = Pipeline([("column_transformer",
                     col_transform),

                    ("random_forest",
                     RandomForestRegressor(max_depth=15,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           random_state=1))])

pipe_rf.fit(X_train, y_train)

In [ ]:
train_preds = pipe_rf.predict(X_train)
test_preds = pipe_rf.predict(X_test)

train_error = np.mean((train_preds - y_train)**2)
test_error = np.mean((test_preds - y_test)**2)


print(f"Качество на трейне: {train_error}")
print(f"Качество на тесте: {test_error}")

Попробуем теперь поэкспериментировать с бэггингами.

Постройте бэггинги с 100 базовыми моделями (и остальными стандартными параметрами) над линейной регрессией, деревом и случайным лесом (бэггинг над бэггингом!).

Какое качество у каждой модели на тесте?

Какой алгоритм получился лучше с точки зрения качества на тесте? (2б)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor

np.random.seed = 1
results = []

for regressor in [BaggingRegressor(LinearRegression(),
                                   n_estimators=100,
                                   n_jobs=-1,
                                   random_state=1),

                  BaggingRegressor(RandomForestRegressor(),
                                   n_estimators=100,
                                   n_jobs=-1,
                                   random_state=1),

                  BaggingRegressor(DecisionTreeRegressor(),
                                   n_estimators=100,
                                   n_jobs=-1,
                                   random_state=1)]:


    pipe_bag = Pipeline([("column_transformer",
                          col_transform),

                         ("bagging",
                          regressor)])

    pipe_bag.fit(X_train, y_train)

    train_preds = pipe_bag.predict(X_train)
    test_preds = pipe_bag.predict(X_test)

    train_error = np.mean((train_preds - y_train)**2)
    test_error = np.mean((test_preds - y_test)**2)

    results.append([train_error, test_error])

In [ ]:
results

Улучшил ли бэггинг над Лесом качество по сравнению с одним Лесом с точки зрения как качества на тесте, так и на трейне. Как это можно объяснить? Как думаете, много ли смысла в использовании бэггинга над линейными моделями? Выбрали бы вы в данной ситуации именно их в качестве базовых?

-- Строить бэггинг над лесом нет смысла, в силу того, что он в отдельности леса оказываются достаточно сильно коррелирующими

-- Строить бэггинг над линейными моделями редко когда может дать хоть какой-то прирост в качестве, потому что линейная модель над линейными моделями - это все еще линейная модель. Но даже если строить нелинейную метамодель, то вступает в силу еще один аргумент - линейные модели обладат низким разбросом и слабо друг от друга отличаются, в построении линейной регрессии отсутствует случайность, поэтому, в целом, их выходы будут максимально близки друг к другу.

### Добавим новые фичи!

Создайте следующие четыре новые вещественные фичи:

1. Отношения площади 1 этажа к общей площади (колонки 1stFlrSF и GrLivArea, в %)
2. Отношение Площади завершенного фундамента первого типа к общей площади фундамента (колонки BsmtFinSF1 и TotalBsmtSF, в %)
3. Возраст дома (между YearBuilt и YrSold)
4. Общая площадь самого дома и фундамента/цоколя (1stFlrSF + 2ndFlrSF + TotalBsmtSF)

Обучите заново Случайный лес и найдите лучшие гиперпараметры на старой сетке.

Улучшили ли качество модели новые фичи? (4б)

In [ ]:
### Реализация Функции для подсчета
### Отношения площади 1 этажа к общей площади

def floor_occupation(x):

    if x["GrLivArea"] == 0:
        return 0
    else:
        return x["1stFlrSF"] * 100 / x["GrLivArea"]


X_train["1stFlrPercent"] = X_train.apply(
    lambda x: floor_occupation(x), axis=1)

X_test["1stFlrPercent"] = X_test.apply(
    lambda x: floor_occupation(x), axis=1)

X["1stFlrPercent"] = X.apply(
    lambda x: floor_occupation(x), axis=1)

In [ ]:
### Реализация Функции для подсчета отношения
### Площади завершенного фундамента первого типа к общей площади фундамента


def bsmt_finish(x):

    if x["TotalBsmtSF"] == 0:
        return 0
    else:
        return x["BsmtFinSF1"] * 100 / x["TotalBsmtSF"]


X_train["BsmtFinPercent"] = X_train.apply(
    lambda x: bsmt_finish(x), axis=1)

X_test["BsmtFinPercent"] = X_test.apply(
    lambda x: bsmt_finish(x), axis=1)

X["BsmtFinPercent"] = X.apply(
    lambda x: bsmt_finish(x), axis=1)

In [ ]:
### Реализация Функции для возраста


def is_house_old(x):

    return int(x['YrSold']) - int(x['YearBuilt'])


X_train["HowOld"] = X_train.apply(
    lambda x: is_house_old(x), axis=1)

X_test["HowOld"] = X_test.apply(
    lambda x: is_house_old(x), axis=1)

X["HowOld"] = X.apply(
    lambda x: is_house_old(x), axis=1)

In [ ]:
### Реализация Функции для общей площади


def total_sq(x):

     return x["1stFlrSF"] + x["2ndFlrSF"] + x['TotalBsmtSF']


X_train["TotalSq"] = X_train.apply(
    lambda x: total_sq(x), axis=1)

X_test["TotalSq"] = X_test.apply(
    lambda x: total_sq(x), axis=1)

X["TotalSq"] = X.apply(
    lambda x: total_sq(x), axis=1)

In [ ]:
numeric_cols_idx = numeric_cols_idx + [X_test.shape[1]-i for i in range(1, 5)]

t = [('OneHotEncoder', OneHotEncoder(), cols_for_ohe_idx),
     ('MeanTargetEncoder', TargetEncoder(), cols_for_mte_idx),
     ('StandardScaler', StandardScaler(), numeric_cols_idx)]

col_transform = ColumnTransformer(transformers=t)

In [ ]:
param_grid = {
    "random_forest__max_depth": [10, 15, 20],
    "random_forest__min_samples_split": [2, 5, 10],
    "random_forest__min_samples_leaf": [1, 3, 5]
}

pipe_rf = Pipeline([("column_transformer",
                     col_transform),

                    ("random_forest",
                     RandomForestRegressor(random_state=1))])

search = GridSearchCV(pipe_rf,
                      param_grid,
                      cv=custom_cv,
                      scoring='neg_mean_squared_error',
                      verbose=10)

search.fit(X, log_target)

print(search.best_params_)

In [ ]:
pipe_rf = Pipeline([("column_transformer",
                     col_transform),

                    ("random_forest",
                     RandomForestRegressor(max_depth=20,
                                           min_samples_leaf=1,
                                           min_samples_split=2))])

pipe_rf.fit(X_train, y_train)

In [ ]:
train_preds = pipe_rf.predict(X_train)
test_preds = pipe_rf.predict(X_test)

train_error = np.mean((train_preds - y_train)**2)
test_error = np.mean((test_preds - y_test)**2)


print(f"Качество на трейне: {train_error}")
print(f"Качество на тесте: {test_error}")

-- Новые фичи немного помогли!